<a href="https://colab.research.google.com/github/YogeshRathee512/Machine-learning-projects/blob/main/topsis%20implementation%20form%20scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer

def calculate_rank(distance_from_best, distance_from_worst):
    return [d_worst / (d_best + d_worst) for d_best, d_worst in zip(distance_from_best, distance_from_worst)]

def euclidean_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

def topsis(input_file, weights, impacts, result_file):
    try:
        # Sample dataset loading (replace with your actual dataset path)
        data = pd.read_csv(input_file)

        # Extract only the numeric columns for normalization
        numeric_columns = data.select_dtypes(include=[np.number]).columns

        # Create an instance of Normalizer
        scaler = Normalizer()

        # Normalize only the numeric columns
        normalized_data = scaler.fit_transform(data[numeric_columns])

        # Replace the original numeric columns with the normalized values
        data[numeric_columns] = normalized_data

        # Display the dataset with normalized values
        data = data.multiply(weights, axis=1)

        count = 0
        ideal_best = []
        ideal_worst = []

        for i in impacts:
            if i == "+":
                ideal_best.append(data.iloc[:, count].max())
                ideal_worst.append(data.iloc[:, count].min())
            else:
                ideal_best.append(data.iloc[:, count].min())
                ideal_worst.append(data.iloc[:, count].max())
            count += 1  # Increment count for the next column

        # Calculate Euclidean distance
        no_of_rows = data.shape[0]
        distance_from_best = []
        distance_from_worst = []

        for i in range(no_of_rows):
            distance_from_best.append(euclidean_distance(data.iloc[i, :], np.array(ideal_best)))
            distance_from_worst.append(euclidean_distance(data.iloc[i, :], np.array(ideal_worst)))

        data["distance_from_best"] = distance_from_best
        data["distance_from_worst"] = distance_from_worst

        arr = calculate_rank(distance_from_best, distance_from_worst)
        data["performace_score"] = arr

        # Create a dictionary to store the ranks
        rank_dict = {value: rank for rank, value in enumerate(sorted(arr, reverse=True), 1)}

        # Use the dictionary to get ranks for each element in the original list
        ranks = [rank_dict[value] for value in arr]
        data["ranks"] = ranks

        # Save the result to a CSV file
        data.to_csv(result_file, index=False)
        print(f"Result saved to {result_file}")

    except FileNotFoundError:
        print("Error: File not found.")
    except pd.errors.EmptyDataError:
        print("Error: Input file is empty.")
    except ValueError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # Check for the correct number of command-line arguments
    if len(sys.argv) != 5:
        print("Usage: python topsis_program.py <InputDataFile> <Weights> <Impacts> <ResultFileName>")
        sys.exit(1)

    # Extract command-line arguments
    input_file = sys.argv[1]
    weights = list(map(float, sys.argv[2].split(',')))
    impacts = sys.argv[3]
    result_file = sys.argv[4]

    # Check for the correct number of weights and impacts
    if len(weights) != len(impacts):
        print("Error: Number of weights and impacts must be the same.")
        sys.exit(1)

    # Convert impacts to numerical values (1 for Benefit, -1 for Cost)
    impacts_numeric = [1 if impact == '+' else -1 for impact in impacts]

    # Call the TOPSIS function
    topsis(input_file, weights, impacts_numeric, result_file)


Your dataset is
         P1        P2        P3        P4        P5
0  0.005043  0.003765  0.027566  0.238683  0.068781
1  0.003274  0.002667  0.017379  0.240478  0.065960
2  0.003771  0.003245  0.024116  0.239408  0.067657
3  0.003503  0.002684  0.019109  0.240227  0.066381
4  0.006911  0.006385  0.042817  0.235117  0.072789
5  0.005095  0.004458  0.020844  0.239712  0.067513
6  0.003318  0.002212  0.015082  0.240802  0.065353
7  0.003220  0.002975  0.014700  0.240802  0.065415
Ideal Best Values:
[0.006910787520323515, 0.002211959571287102, 0.042816835723743514, 0.23511701020231088, 0.07278862073036398]
Ideal Worst Values:
[0.0032200239248666436, 0.006384966730733682, 0.014700109222217286, 0.24080196241966406, 0.0653533509698462]
Distance from Best:
[0.01634813119244091, 0.02712726093687753, 0.020134227514322138, 0.025319312029186393, 0.0041730071594465795, 0.02323939082959648, 0.029491642282348386, 0.0298568942880921]
Distance from Worst:
[0.013854996153342814, 0.00463379190189283, 0